# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В нашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Мы будем строить модели для определения стоимости подержанного автомобиля,  попутно оценивая такие параметры модели как:
- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных
В нашем распоряжении файл данных с характеристиками продаваемых автомобилей и ценами:
- DateCrawled — дата скачивания анкеты из базы
- VehicleType — тип автомобильного кузова
- RegistrationYear — год регистрации автомобиля, 
- Gearbox — тип коробки передач
- Power — мощность (л. с.)
- Model — модель автомобиля
- Kilometer — пробег (км)
- RegistrationMonth — месяц регистрации автомобиля
- FuelType — тип топлива
- Brand — марка автомобиля
- Repaired — была машина в ремонте или нет
- DateCreated — дата создания анкеты
- NumberOfPictures — количество фотографий автомобиля
- PostalCode — почтовый индекс владельца анкеты (пользователя)
- LastSeen — дата последней активности пользователя
- Price - цена автомобиля в евро

In [3]:
import pandas as pd
import statistics as st
import warnings
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
warnings.filterwarnings('ignore')

In [4]:
try:
    data = pd.read_csv('/datasets/autos.csv')
except:
    data = pd.read_csv('C:\\YP\\autos.csv')
data.info()
display(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  Repaired           283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,Repaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


Проверим данные на дубликаты.

In [5]:
data = data.drop_duplicates().reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354365 entries, 0 to 354364
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354365 non-null  object
 1   Price              354365 non-null  int64 
 2   VehicleType        316875 non-null  object
 3   RegistrationYear   354365 non-null  int64 
 4   Gearbox            334532 non-null  object
 5   Power              354365 non-null  int64 
 6   Model              334660 non-null  object
 7   Kilometer          354365 non-null  int64 
 8   RegistrationMonth  354365 non-null  int64 
 9   FuelType           321470 non-null  object
 10  Brand              354365 non-null  object
 11  Repaired           283211 non-null  object
 12  DateCreated        354365 non-null  object
 13  NumberOfPictures   354365 non-null  int64 
 14  PostalCode         354365 non-null  int64 
 15  LastSeen           354365 non-null  object
dtypes: int64(7), object(

Маловероятно, чтобы такие параметры как дата создания объявления/скачивания объявления, месяц регистрации, дата последнего визита владельца, индекс владельца и количество фото автомобиля влияли на итоговую цену автомобиля, поэтому мы удалим столбцы, содержание эти данные.

In [6]:
data = data.drop(columns = ['DateCrawled', 'DateCreated', 'PostalCode', 'RegistrationMonth', 'NumberOfPictures', 'LastSeen'], axis = 1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354365 entries, 0 to 354364
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             354365 non-null  int64 
 1   VehicleType       316875 non-null  object
 2   RegistrationYear  354365 non-null  int64 
 3   Gearbox           334532 non-null  object
 4   Power             354365 non-null  int64 
 5   Model             334660 non-null  object
 6   Kilometer         354365 non-null  int64 
 7   FuelType          321470 non-null  object
 8   Brand             354365 non-null  object
 9   Repaired          283211 non-null  object
dtypes: int64(4), object(6)
memory usage: 27.0+ MB


Обработаем данные следующим образом: 
- удалим заведомые выбросы (выбросы найдены в столбце `RegistrationYear`)
- будем предполагать, что автомобиль по умолчанию не ремонтировался
- пропуске в столбцах `VehicleType`, `Gearbox`, `Model`, `FuelType` заполним модовым значением по бренду
- в графе `Power` удалим явно завышенные значения (их немного), а для нулевых и малых значений (таких данных большое количество) заполним медианным значением мощности по бренду, поскольку медиана не слишком чувствительна к выбросам
- оставшиеся данные с пропусками удалим

Мы отдаём себе отчёт, что предобработка выполнена недостаточно чисто, а кроме того может быть просачивание данных их тренировочной выборки в валидационную и тестовую, но всё же сделаем так, поскольку данные изначально даны очень грязные, и если оставить только корректные данные, то это кратно уменьшит датасет, при этом любая предобработка всё равно будет содержать значительные допущения.

In [8]:
data.groupby('Repaired')['Repaired'].count()

Repaired
no     247158
yes     36053
Name: Repaired, dtype: int64

In [9]:
data['Repaired'] = data['Repaired'].fillna('no')


In [10]:
brands = data['Brand'].unique()

In [11]:
for brand in brands:
    mode_fuel_type = st.mode(data[(data['Brand']==brand)]['FuelType'])
    data.loc[(data['Brand']==brand)&(data['FuelType'].isna()), 'FuelType'] =  mode_fuel_type

In [12]:
for brand in brands:
    mode_model = st.mode(data[data['Brand']==brand]['Model'])
    data.loc[(data['Brand']==brand)&(data['Model'].isna()),'Model'] = mode_model

In [13]:
for brand in brands:
    mode_gearbox = st.mode(data[data['Brand']==brand]['Gearbox'])
    data.loc[(data['Brand']==brand)&(data['Gearbox'].isna()),'Gearbox'] = mode_gearbox

In [14]:
for brand in brands:
    mode_gearbox = st.mode(data[data['Brand']==brand]['VehicleType'])
    data.loc[(data['Brand']==brand)&(data['VehicleType'].isna()),'VehicleType'] = mode_gearbox

In [15]:
print(data['RegistrationYear'].describe())
data = data.query('(RegistrationYear > 1980) & (RegistrationYear <= 2016)')


count    354365.000000
mean       2004.234481
std          90.228466
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: RegistrationYear, dtype: float64


In [16]:
data['Kilometer'].describe()


count    335795.000000
mean     128576.557126
std       37405.568445
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: Kilometer, dtype: float64

In [17]:
data['Power'].describe()
data = data.query('(Power <= 2000)')

In [19]:
for brand in brands:
    median_power = data[data['Brand']==brand]['Power'].median()
    data.loc[(data['Brand']==brand)&(data['Power']<100),'Power'] = median_power

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 335688 entries, 0 to 354364
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             335688 non-null  int64 
 1   VehicleType       334958 non-null  object
 2   RegistrationYear  335688 non-null  int64 
 3   Gearbox           335688 non-null  object
 4   Power             335688 non-null  int64 
 5   Model             333200 non-null  object
 6   Kilometer         335688 non-null  int64 
 7   FuelType          335688 non-null  object
 8   Brand             335688 non-null  object
 9   Repaired          335688 non-null  object
dtypes: int64(4), object(6)
memory usage: 28.2+ MB


In [21]:
data = data.dropna(subset=['VehicleType', 'Model'])
data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 333200 entries, 0 to 354364
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             333200 non-null  int64 
 1   VehicleType       333200 non-null  object
 2   RegistrationYear  333200 non-null  int64 
 3   Gearbox           333200 non-null  object
 4   Power             333200 non-null  int64 
 5   Model             333200 non-null  object
 6   Kilometer         333200 non-null  int64 
 7   FuelType          333200 non-null  object
 8   Brand             333200 non-null  object
 9   Repaired          333200 non-null  object
dtypes: int64(4), object(6)
memory usage: 28.0+ MB


## Обучение моделей

Мы имеем дело с задачей регрессии, поэтому рассмотрим простую модель регрессии (DecisionTreeRegressor) и модели, использующие бустинги (CatBoost и LightGBM).

In [22]:
features =  data.drop(['Price'], axis = 1)
target = data['Price']

In [23]:
features_train_valid, features_test, target_train_valid, target_test = train_test_split(
    features, target, test_size=0.2, random_state=12345)

features_train, features_valid, target_train, target_valid = train_test_split(
    features_train_valid, target_train_valid, test_size=0.25, random_state=12345)

print(target_train.shape, target_valid.shape, target_test.shape)
print(features_train.shape, features_valid.shape, features_test.shape)

(199920,) (66640,) (66640,)
(199920, 9) (66640, 9) (66640, 9)


### Простые модели

В качестве простой модели мы возьмём дерево решений. По существу, все оставшиеся признаки категориальны, даже если их и можно сравнить по порядку, поэтому DecisionTreeRegressor можно считать хорошим выбором.
Дерево решений работает с категориальными признаками, переведённые в числовой формат, поэтому будем использовать порядковое кодирование. Подбор гиперпараметров осуществим с помощью GreedSearchCV.

In [24]:
features_ordinal_train = features_train.copy()
features_ordinal_valid = features_valid.copy()
features_ordinal_test = features_test.copy()

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 333200 entries, 0 to 354364
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             333200 non-null  int64 
 1   VehicleType       333200 non-null  object
 2   RegistrationYear  333200 non-null  int64 
 3   Gearbox           333200 non-null  object
 4   Power             333200 non-null  int64 
 5   Model             333200 non-null  object
 6   Kilometer         333200 non-null  int64 
 7   FuelType          333200 non-null  object
 8   Brand             333200 non-null  object
 9   Repaired          333200 non-null  object
dtypes: int64(4), object(6)
memory usage: 28.0+ MB


In [27]:
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']
encoder = OrdinalEncoder()
encoder.fit(features_ordinal_train[cat_features])

OrdinalEncoder()

In [28]:
features_ordinal_train[cat_features] = encoder.transform(features_ordinal_train[cat_features])

features_ordinal_valid[cat_features] = encoder.transform(features_ordinal_valid[cat_features])

features_ordinal_test[cat_features] = encoder.transform(features_ordinal_test[cat_features])                                                                                        

In [29]:
dtr = DecisionTreeRegressor(random_state = 12345)
grid = {'min_samples_split': (2, 5), 'max_depth': (1,10)}

gdtr = GridSearchCV(dtr, param_grid=grid, scoring=mean_squared_error, n_jobs=-1)
gdtr.fit(features_ordinal_train, target_train)
best_score = gdtr.best_score_
best_model = gdtr.best_estimator_
best_params = gdtr.best_params_

In [30]:
%%time
best_model.fit(features_ordinal_train, target_train)

CPU times: total: 46.9 ms
Wall time: 46.8 ms


DecisionTreeRegressor(max_depth=1, random_state=12345)

In [31]:
%%time
predictions_valid_dtr = best_model.predict(features_ordinal_valid)
print('RMSE для дерева решений: ', mean_squared_error(target_valid, predictions_valid_dtr)**0.5)

RMSE для дерева решений:  3609.4220504848904
CPU times: total: 0 ns
Wall time: 0 ns


 ### Модели, использующие бустинг
Мы рассмотрим 2 модели:  CatBoost и LightGBM. Обе эти модели имеют высокую скорость, а кроме того работают с категориальными признаками напрямую, без необходимости их предварительно обрабатывать, поэтому мы будем использовать выборки до применения порядкового кодирования. Отметим, что поскольку модели градиентного бустинга обучаются достаточно долго, а кроме того по умолчанию многие параметры имеют максимальное значение (в частности, количество деревьев или глубина дерева в CatBoost), то мы не будем подбирать параметры автоматически, попутно кроссвалидируя, вместо этого разобьём выборку на тренировочную, валидационную и тестовую части. Кроме того, мы нормировали количественные данные (на итоговый результат это никак не повлило).


In [32]:
quant_features = ['RegistrationYear', 'Power', 'Kilometer']

scaler = StandardScaler().fit(features_train[quant_features])

features_train[quant_features] = scaler.transform(features_train[quant_features])
features_valid[quant_features] = scaler.transform(features_valid[quant_features])
features_test[quant_features] = scaler.transform(features_test[quant_features])

#### CatBoost

In [34]:
%%time
cat_features = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'Repaired']

cbr = CatBoostRegressor(loss_function="RMSE", iterations=100, depth = 16) 

cbr.fit(features_train, target_train, cat_features = cat_features, verbose=False)



CPU times: total: 3min 8s
Wall time: 1min 37s


In [35]:
%%time
predictions_valid = cbr.predict(features_valid)
print('RMSE для CatBoost на валидационной выборке', mean_squared_error(target_valid, predictions_valid)**0.5)


RMSE для CatBoost на валидационной выборке 1706.3384776182102
CPU times: total: 141 ms
Wall time: 126 ms


#### LightGBM

In [36]:
for feature in cat_features:
    features_train[feature] = features_train[feature].astype('category')
    features_valid[feature] = features_valid[feature].astype('category')
    features_test[feature] = features_test[feature].astype('category')
    

In [37]:
%%time
lgbr = LGBMRegressor(n_estimators = 100, n_jobs=-1)
lgbr.fit(features_train, target_train, verbose=False)



CPU times: total: 3.11 s
Wall time: 487 ms


LGBMRegressor()

In [38]:
%%time
predictions_valid = lgbr.predict(features_valid)

print('RMSE для LightGBM на валидационной выборке', mean_squared_error(target_valid, predictions_valid)**0.5)

RMSE для LightGBM на валидационной выборке 1725.1084309644168
CPU times: total: 734 ms
Wall time: 110 ms


## Анализ моделей и итоговое тестирование

Нами были рассмотрены 3 модели: дерево решений, бустинг с использованием библиотеки Catboost и бустинг с использованием LightGBM. Результаты работы (время обучения, время предсказания на валидационной выборке, RMSE на валидационной выборке) можно увидеть в таблице ниже. Дерево решений, хотя и быстро работает, но не даёт целевого значения RMSE, поэтому не подходит для итогового тестирования. Обе модели с ипользованием библиотек бустинга дают целевое значение RMSE, при этом CatBoost работаетточнее, поэтому для итогового тестирование выберем модель CatBoost.

Итоговое тестирование пройдено успешно, RMSE очень близко к значению на валидационной выборке.

In [39]:
result = pd.DataFrame(data = [['DecisionTreeRegressor','47 ms','0 ms', 3609],
                              ['CatBoost', '1min 37s' ,'126 ms', 1706],
                              ['LightGBM','487ms','110 ms', 1725]], columns = ['Model', 'FitTime ', 'PredTime', 'RMSE_valid'])

In [40]:
display(result)

,Model,FitTime,PredTime,RMSE_valid
0,DecisionTreeRegressor,47 ms,0 ms,3609
1,CatBoost,1min 37s,126 ms,1706
2,LightGBM,487ms,110 ms,1725


In [41]:
%%time
predictions_test = cbr.predict(features_test)
print('RMSE для Catboost на тестовой выборке', mean_squared_error(target_test, predictions_test)**0.5)

RMSE для Catboost на тестовой выборке 1706.2318083262446
CPU times: total: 172 ms
Wall time: 76.5 ms
